# Drone Model-Predictive Control

## Package Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, FindResourceOrThrow, SceneGraph,
    Parser, Simulator, VectorSystem, plot_system_graphviz, MeshcatVisualizerCpp, StartMeshcat)

from pydrake.examples.quadrotor import (QuadrotorGeometry, QuadrotorPlant, StabilizingLQRController)

from underactuated.meshcat_cpp_utils import MeshcatSliders



In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://7a2da9b8-cd38-4703-b181-4b0e0b197067.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


## Trajectory Optimization for the Quadrotor

Find an optimal trajectory from an initial condition to a target destination, that minimizes 

## Wire Up Block Diagram &amp; Run simulation

In [4]:
def simulate_quadrotor():

    #define diagram builder, used to connect plant and controller inputs / outputs
    builder = DiagramBuilder()


    quadrotor_plant = QuadrotorPlant()

    plant = builder.AddSystem(quadrotor_plant)

    controller = builder.AddSystem(StabilizingLQRController(plant, [1, 2, 2]))
    builder.Connect(controller.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), controller.get_input_port(0))

    # Set up visualization in MeshCat
    scene_graph = builder.AddSystem(SceneGraph())
    QuadrotorGeometry.AddToBuilder(builder, plant.get_output_port(0), scene_graph)
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty('/Background','visible',False)
    MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    # end setup for visualization



    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()



    #continuous_state = context.get_mutable_continuous_state_vector()

    # Set the initial conditions (theta1, theta2, theta1dot, theta2dot)
    #context.SetContinuousState([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    context.SetContinuousState(0.5*np.random.randn(12,))

   # print(f"continuous state {continuous_state}")
    #print(f"size of continuous state {continuous_state.size()}")
    simulator.set_target_realtime_rate(1.0)
    meshcat.AddButton('Stop Simulation')
    while meshcat.GetButtonClicks('Stop Simulation') < 1:
        
        simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)

    meshcat.DeleteAddedControls()

    print("simulation complete")

simulate_quadrotor()

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7a2da9b8-cd38-4703-b181-4b0e0b197067' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>